# Start

In [1]:
import tensorflow as tf

2025-01-13 18:22:35.068352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736803355.086742 2821637 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736803355.092044 2821637 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 18:22:35.112047: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Functions

In [2]:
# Parsing function.
def parse_example(example_proto, feature_description):
    # Parse the example.
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)

    # Convert SparseTensors to DenseTensors.
    #   This is necessary because we loaded features with variable length.
    #   Although the parsed files have the desired data, they have other things 
    #   (indices, values and dense_shape). Neural networks must work with values
    #   in a more direct way.
    dense_features = {
        key: tf.sparse.to_dense(value, default_value=0.0)
        if isinstance(value, tf.SparseTensor) else value
        for key, value in parsed_features.items()
    }

    return dense_features

In [3]:
def read_tfrecord(file, batch_size = 10):
    # Define feature description for parsing.
    #   Comment every feature that will not be used.
    known_feature_description = {
        # 'NOISY_ALBEDO_B-NIR' : tf.io.VarLenFeature(tf.float32),
        # 'NOISY_ALBEDO_B-UV' : tf.io.VarLenFeature(tf.float32),
        # 'NOISY_ALBEDO_B-Vis' : tf.io.VarLenFeature(tf.float32),
        # 'NOISY_ALBEDO_SS-NIR' : tf.io.VarLenFeature(tf.float32),
        # 'NOISY_ALBEDO_SS-UV' : tf.io.VarLenFeature(tf.float32),
        # 'NOISY_ALBEDO_SS-Vis' : tf.io.VarLenFeature(tf.float32),

        'NOISY_ALBEDO_B-NIR_norm' : tf.io.VarLenFeature(tf.float32),
        'NOISY_ALBEDO_B-UV_norm' : tf.io.VarLenFeature(tf.float32),
        'NOISY_ALBEDO_B-Vis_norm' : tf.io.VarLenFeature(tf.float32),
        
        'NOISY_ALBEDO_SS-NIR_norm' : tf.io.VarLenFeature(tf.float32),
        'NOISY_ALBEDO_SS-UV_norm' : tf.io.VarLenFeature(tf.float32),
        'NOISY_ALBEDO_SS-Vis_norm' : tf.io.VarLenFeature(tf.float32),

        'radius' : tf.io.FixedLenFeature([], tf.float32),
        'log_gravity' : tf.io.FixedLenFeature([], tf.float32),
        'log_temperature' : tf.io.FixedLenFeature([], tf.float32),
        'log_pressure' : tf.io.FixedLenFeature([], tf.float32),

        # 'OBJECT-DIAMETER' : tf.io.FixedLenFeature([], tf.float32),
        # 'OBJECT-GRAVITY' : tf.io.FixedLenFeature([], tf.float32),
        # 'ATMOSPHERE-TEMPERATURE' : tf.io.FixedLenFeature([], tf.float32),
        # 'ATMOSPHERE-PRESSURE' : tf.io.FixedLenFeature([], tf.float32),
        
        'Earth_type' : tf.io.FixedLenFeature([], tf.string),

        'log_C2H6' : tf.io.FixedLenFeature([], tf.float32),
        'log_CH4' : tf.io.FixedLenFeature([], tf.float32),
        'log_CO' : tf.io.FixedLenFeature([], tf.float32),
        'log_CO2' : tf.io.FixedLenFeature([], tf.float32),
        'log_H2O' : tf.io.FixedLenFeature([], tf.float32),
        'log_N2' : tf.io.FixedLenFeature([], tf.float32),
        'log_N2O' : tf.io.FixedLenFeature([], tf.float32),
        'log_O2' : tf.io.FixedLenFeature([], tf.float32),
        'log_O3' : tf.io.FixedLenFeature([], tf.float32)
    }

    # Read TFRecord.
    dataset = tf.data.TFRecordDataset(file)
    parsed_dataset = dataset.map(lambda x: parse_example(x, known_feature_description), num_parallel_calls=tf.data.AUTOTUNE)

    parsed_dataset = parsed_dataset.batch(batch_size, drop_remainder=False)
    parsed_dataset = parsed_dataset.prefetch(tf.data.AUTOTUNE)
    parsed_dataset = tf.data.Dataset.range(1).interleave(lambda _: parsed_dataset, num_parallel_calls=tf.data.AUTOTUNE)

    return parsed_dataset

# Run

In [4]:
tfrecord_file = '/home/cleber/Documentos/Sarah/geexhp-main/data/TFRecord_data/proterozoic_data27_66660-73326_3130.tfrecord'

train_ds = read_tfrecord(tfrecord_file)

I0000 00:00:1736803357.343609 2821637 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 707 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:08:00.0, compute capability: 7.5


In [5]:
count = 0

for batch in train_ds:
    count += 1

count

2025-01-13 18:22:37.476720: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
2025-01-13 18:22:37.558452: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


313

In [6]:
len(batch)

20

In [9]:
batch.keys()

dict_keys(['NOISY_ALBEDO_B-NIR_norm', 'NOISY_ALBEDO_B-UV_norm', 'NOISY_ALBEDO_B-Vis_norm', 'NOISY_ALBEDO_SS-NIR_norm', 'NOISY_ALBEDO_SS-UV_norm', 'NOISY_ALBEDO_SS-Vis_norm', 'Earth_type', 'log_C2H6', 'log_CH4', 'log_CO', 'log_CO2', 'log_H2O', 'log_N2', 'log_N2O', 'log_O2', 'log_O3', 'log_gravity', 'log_pressure', 'log_temperature', 'radius'])